<a href="https://colab.research.google.com/github/tjturnage/radar/blob/main/Plot_NEXRAD_mosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Plot NEXRAD Reflectivity Mosaics**<br />

This allows the user to plot nexrad mosaic reflectivity images described at:
https://mesonet.agron.iastate.edu/docs/nexrad_mosaic/
<br />
<br />
The user can define the following:
- lat/lon box containing the plot
- one of two available reflectivity colormaps (additional maps available on request)
- which cities to plot (if any) and where to place the labels
- whether to plot major roads
- whether to plot county outlines
<br/> <br/>

When the script completes two files will be created:
- radar_images.tar
- radar_images.zip


Either (or both) of these files can be downloaded and then extracted.
<br />
<br />   
  
&nbsp;&nbsp;*T.J. Turnage (NWS Grand Rapids, MI - thomas.turnage@noaa.gov)*
<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*09-Feb-2024 -- v1.00*<br/>


In [1]:
# @title Install the needed GIS software
!pip install rasterio &> /dev/null
!pip install cartopy &> /dev/null

In [13]:
# @title

#############################################################
# Configuration Section
#############################################################

# This is in UTC time, must be 5 minute increments
# Example: 20231031_1630 = 31 October 2023 16:30 UTC
first_plot_time = "20230824_2245"
last_plot_time = "20230825_0245"

# image time step in minutes. Must be multiple of 5
# For example, a value of 15 plots every 15 minutes
time_step = 5

# Define your local UTC time shift and observed time zone
utc_minus_local_time = 0
time_zone = "UTC"

# plot lat/lon ranges
lat_min = 42.0
lat_max = 44.25
lon_min = -86.9
lon_max = -83.95

# reflectivity colormap selection
plot_cmap = "wdtd_z"  # options are "wdtd_z" and "winter_z"

# This will require experimentation on your part because the combination of
# your chosen lat/lon ranges and figure dimensions often result in some
# undesired map stretching. plot_aspect_ratio can also help with adjustments.
# plot_aspect_ratio > 1 shrinks horizontal scale and vice versa.

fig_height = 7
fig_width = 5
plot_aspect_ratio = 1.25

###  ** You'll need to create your own cities dictionary ** ###
#   You'll want to test the best offset position for each of the labels.
#   Positive "lat_offset" shifts the label north by that many degrees
#   Negative "lon_offest" shifts the label east by that many degrees

plot_counties = True
plot_roads = True
plot_cities = True

create_animation = False

# You'll need to create your own cities dictionary for your area
cities = {
'Grand Rapids':{'lat':42.963,'lon':-85.668, 'lat_offset':0.04, 'lon_offset':0.06},
'Muskegon':{'lat':43.234,'lon':-86.248, 'lat_offset':0.04, 'lon_offset':0.06},
'Holland':{'lat':42.7875,'lon':-86.1089, 'lat_offset':0.04, 'lon_offset':0.06},
'South Haven':{'lat':42.4031,'lon':-86.2736, 'lat_offset':0.04, 'lon_offset':0.06},
'Lansing':{'lat':42.7325,'lon':-84.5556, 'lat_offset':0.00, 'lon_offset':0.06},
'Kalamazoo':{'lat':42.292,'lon':-85.587, 'lat_offset':-0.10, 'lon_offset':0.06},
'Ludington':{'lat':43.955,'lon':-86.4525, 'lat_offset':0.04, 'lon_offset':0.06},
'Big Rapids':{'lat':43.698,'lon':-85.4836, 'lat_offset':0.04, 'lon_offset':0.06},
'Mount Pleasant':{'lat':43.5978,'lon':-84.7675, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC109':{'lat':43.0534,'lon':-85.9080, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC158':{'lat':43.0923,'lon':-86.0977, 'lat_offset':0.06, 'lon_offset':0.06},
#'MC159':{'lat':43.4101,'lon':-86.3146, 'lat_offset':0.06, 'lon_offset':0.06},
           }

#############################################################
# End Configuration Section
#############################################################

In [14]:
# @title Ensure colormaps and GIS files are ready
import sys
import os
import subprocess
import glob
from os.path import exists
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import urllib.request
import requests
from bs4 import BeautifulSoup
import zipfile
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.io.shapereader as shpreader
import xarray as xr
from datetime import datetime
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

os.chdir('/content')

def create_and_register_cmap(cmap_name, original_colors, position, rgb_input=True):
    """
    1) Converts list of color values from 8-bit (0-255 scale)
       to 0-1 decimal scale.

    2) The created final_color_list is combine with position list
       to create a matplotlib colormap.

    3) Registers the new colormap using the supplied cmap_name.

    Input
    ----------
             cmap_name : string
                         Name of the cmap you want to use and register

      original_ colors : list
                         8-bit tuples containing RGB values
                         example: [(127,256,192), (125,125,125)]

              position : list
                         node locations for color break points on
                         a 0 to 1 scale

            rgb_input : boolean
                        True: Color conversion from RGB to performed.
                       False: Color conversion from RGB is skipped.

    Returns
    -------
    None

    """
    if rgb_input:
        final_color_list = []
        for color in original_colors:
            converted_color_group = []
            for original_channel_value in color:
                new_channel_value = float(original_channel_value)/255
                converted_color_group.append(float(format(new_channel_value, '.3f')))
            converted_color_tuple = tuple(converted_color_group)
            final_color_list.append(converted_color_tuple)
    else:
        final_color_list = original_colors

    this_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(position, final_color_list)))
    mpl.colormaps.register(cmap=this_cmap, force=True)
    return


###############################################################################

winter_rgbs = [(0, 0, 0), (70, 70, 70), (210, 210, 210),
(0, 120, 255), (0, 0, 60), (225, 225, 0), (220, 100, 0), (220, 100, 0)]

winter_nodes = [0.0, 0.20, 0.38, 0.42, 0.48, 0.52, 0.60, 1.0]

create_and_register_cmap('winter_z', winter_rgbs, winter_nodes)

#------------------------------------------------------------------------------

wdtd_z_rgbs =[(0, 0, 0),
 (0, 0, 0), (130, 130, 130),
 (95, 189, 207), (57, 201, 105),
 (57, 201, 105), (0, 40, 0),
 (9, 94, 9), (255, 207, 0),
 (255, 207, 0), (255, 133, 0),
 (255, 0, 0), (89, 0, 0),
 (255, 245, 255), (225, 11, 227),
 (164, 0, 247), (99, 0, 214),
 (5, 221, 224), (58, 103, 181),
 (255, 255, 255)]

wdtd_z_nodes = [0.0,
                0.0, 44.99/110,
                45/110, 49.99/110,
                50/110, 64.99/110,
                65/110, 69.99/110,
                70/110, 79.99/110,
                80/110, 89.99/110,
                90/110, 99.99/110,
                100/110, 104.99/110,
                105/110, 109.99/110,
                110/110]

create_and_register_cmap('wdtd_z', wdtd_z_rgbs, wdtd_z_nodes)

###############################################################################

#scan this webpage to determine the most recent NWS County shapefile available
url = "https://www.weather.gov/gis/Counties"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
zip_links = soup.find_all("a", href=lambda href: href.endswith(".zip"))
if zip_links:
    zip_url = zip_links[-1]["href"]
    counties_url = os.path.join('https://www.weather.gov',str(zip_url)[1:])
    zip_file_name = counties_url.split("/")[-1]
    zip_file_root = zip_file_name[:-4]
    shapefile_name = f'{zip_file_root}.shp'
else:
    print("No zip files found on the website.")

if not exists(zip_file_name):
  urllib.request.urlretrieve(counties_url, "counties.zip")
  with zipfile.ZipFile('counties.zip', 'r') as zip_ref:
      zip_ref.extractall('.')

try:
  COUNTY_SHAPEFILE
except NameError:
  #reader = shpreader.Reader("tl_2019_us_county.shp")
  reader = shpreader.Reader(shapefile_name)
  features = list(reader.geometries())
  COUNTY_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

# download and read road shapefile only if plot_roads=True
if plot_roads:
  road_shapefile_name = "tl_2019_us_primaryroads.shp"
  road_url = "https://www2.census.gov/geo/tiger/TIGER2019/PRIMARYROADS/tl_2019_us_primaryroads.zip"
  if not exists(road_shapefile_name):
    urllib.request.urlretrieve(road_url, "roads.zip")
    with zipfile.ZipFile('roads.zip', 'r') as zip_roads:
      zip_roads.extractall('.')
  try:
    ROAD_SHAPEFILE
  except NameError:
    #reader = shpreader.Reader("tl_2019_us_primary_roads.shp")
    road_reader = shpreader.Reader(road_shapefile_name)
    features = list(road_reader.geometries())
    ROAD_SHAPEFILE = cfeature.ShapelyFeature(features, ccrs.PlateCarree())

# for these pngs, we already know their values of lat/lon, ranges, dx, dy
x = np.arange(-126.0, -65.0, 0.005)
y = np.arange(23.0, 50.0, 0.005)
coords = {'longitude': x, 'latitude': y}

plt.rcParams['font.size'] = 10
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['lines.markersize'] = 6
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['text.color'] = 'white'
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['scatter.marker'] = 'o'

In [ ]:
# @title Create your images!

def create_valid_times(initial_time,final_time):
  if int(initial_time[-2:])%5 != 0:
    print("Initial time must be to nearest 5 minutes!")

  start_time = datetime.strptime(initial_time, "%Y%m%d_%H%M")
  end_time = datetime.strptime(final_time, "%Y%m%d_%H%M")

  valid_times = []
  i = 0
  this_time = start_time + timedelta(minutes=time_step*i)
  valid_times.append(this_time)
  while this_time < end_time:
    i += 1
    this_time = start_time + timedelta(minutes=time_step*i)
    valid_times.append(this_time)

  return valid_times

valid_times = create_valid_times(first_plot_time,last_plot_time)

# ------------- Download pngs, create plots, save images -------------

PNG_URL_BASE = "https://mesonet.agron.iastate.edu/archive/data"

os.chdir('/content')
# create fresh images directory to store new images to
result = subprocess.run(["rm -rf images ; mkdir images"], shell=True, capture_output=True, text=True)
fout = open('output.txt', 'w')
fout.write(f'dts,station,ref\n')
for plot_time in valid_times:
  local_time = plot_time - timedelta(hours=utc_minus_local_time)

  tail = datetime.strftime(plot_time,"%Y/%m/%d/GIS/uscomp/n0q_%Y%m%d%H%M.png")
  downloaded_file_name = tail[-16:]
  if not exists(downloaded_file_name):
    url = os.path.join(PNG_URL_BASE,tail)
    map = urllib.request.urlretrieve(url,downloaded_file_name)

  with rasterio.open(downloaded_file_name) as src:
    # Get the image data and metadata
    image_data = src.read()

    #slice (x,y) cartesian array out of current (1,x,y) array
    image_new = image_data[0,:,:]

    # convert 8-bit (0-255) values to reflectivity values as described at
    # https://mesonet.agron.iastate.edu/GIS/rasters.php?rid=2
    image_new = (image_new * 0.5) - 32.5

    #flip data vertically to move origin from upper left to lower left
    image_flipped = np.flipud(image_new)

    # Add data to xarray with the defined coordinate system
    data = xr.DataArray(image_flipped, coords=coords, dims=['latitude', 'longitude'])
    data.attrs["units"] = "dBZ"
    data.attrs["long_name"] = "Reflectivity"

    # Determine region you want to plot
    subset = data.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

    fig, ax = plt.subplots(1,1,figsize=(fig_height,fig_width),subplot_kw={'projection': ccrs.PlateCarree()})
    if plot_cmap == "wdtd_z":  # options are "wdtd_z" and "winter_z"
      subset.plot(ax=ax, cmap=plot_cmap, vmin=-32, vmax=95)
    else:
      subset.plot(ax=ax, cmap=plot_cmap, vmin=-20, vmax=80)

    ax.set_aspect(plot_aspect_ratio)
    if plot_counties:
      ax.add_feature(COUNTY_SHAPEFILE, facecolor='none', edgecolor='#cccccc', linewidth=0.7)

    if plot_roads:
      ax.add_feature(ROAD_SHAPEFILE, facecolor='none', edgecolor='cyan', linewidth=0.5)
    ax.add_feature(cfeature.STATES, facecolor='none', edgecolor='white', linewidth=1.3)
    ax.set_title(datetime.strftime(local_time,f'%b %d, %Y -- %I:%M %p {time_zone}'), color='black')
    ax.set_title(datetime.strftime(local_time,f'%b %d, %Y -- %H:%M UTC'), color='black')
    plt.tight_layout()
    target_fname = f'images/cropped_{downloaded_file_name}'
    if plot_cities:
      for key in cities:
        c = cities[key]
        lat,lon = c['lat'], c['lon']
        if ((lat > lat_min) & (lat < lat_max) & (lon > lon_min) & (lon < lon_max)):
          if 'MC' in key:
            plt.plot([lon], [lat], color='#FFAAAA', linewidth=1, marker='o', markersize=4, zorder=10)
            plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='#FFAAAA', zorder=11)
          else:
            plt.plot([lon], [lat], color='#FFFF00', linewidth=1, marker='o', markersize=6, zorder=10)
            plt.plot([lon], [lat], color='white', linewidth=1, marker='o', markersize=4, zorder=10)
            plt.text(lon + c['lon_offset'], lat + c['lat_offset'], key, color='#FFFFFF', zorder=11)
    plt.savefig(target_fname,bbox_inches='tight')


import glob
import os

if create_animation:
  !apt install imagemagick &> /dev/null
  os.chdir('/content/images')
  cmd = 'convert -delay 10 -loop 0 *.png animation.gif'
  os.system(cmd)

# tar images files together so you can download single file
os.chdir('/content/')
with zipfile.ZipFile('radar_images.zip', 'w') as f:
    for file in glob.glob('images/*'):
        f.write(file)

!tar cvf /content/radar_images.tar /content/images/cropped*


In [ ]:
!rm /content/*png
    'MC109':{'lat':43.0534,'lon':-85.9080, 'lat_offset':0.06, 'lon_offset':0.06},
    'MC158':{'lat':43.0923,'lon':-86.0977, 'lat_offset':0.06, 'lon_offset':0.06},
    'MC159':{'lat':43.4101,'lon':-86.3146, 'lat_offset':0.06, 'lon_offset':0.06}

  dx = 0.01
  dts = datetime.strftime(plot_time,"%Y-%m-%d %H:%M:%SZ")
  for stn in ('MC109','MC158','MC159'):
    lat = cities[stn]['lat']
    lon = cities[stn]['lon']
    subset = data.sel(latitude=slice(lat - dx, lat + dx), longitude=slice(lon - dx, lon + dx))
    ref = np.mean(subset.data)
    fout.write(f'{dts},{stn},{ref}\n')

fout.close()

# @title Ignore this code
def map_ref(z):
  z_new = ((z + 32) / 122)
  return r

mc109 = df[df['station'] == 'MC109']['ref']
#print(list(mc109.index))

mc158 = list(df[df['station'] == 'MC158']['ref'])
mc159 = list(df[df['station'] == 'MC159']['ref'])

d = {'MC109': mc109, 'MC158': mc158, 'MC159': mc159}
mean_ref = pd.DataFrame(data=d, index=mc109.index)
#print(mean_ref)
plot = mean_ref.plot()


import pandas as pd
#print(data.data)
df = pd.read_csv('output.txt')
df['date'] = pd.DatetimeIndex(df['dts'])
df.set_index('date', inplace=True)
df.drop('dts',axis=1,inplace=True)
print(df.index)

mean_ref.to_csv('ref.csv', index=True)

cmap_dict = {}
with open('TWIP_BR_test.pal', 'r') as fin:
  for line in fin:
    if 'Color' in line:
      values = line.split()
      ref = float(values[1])
      norm_ref = map_ref(ref)
      rgb_slice = [int(c) for c in values[2:5]]
      rgb_tuple = tuple(rgb_slice)
      cmap_dict[norm_ref] = rgb_tuple

sorted_keys = sorted(list(cmap_dict.keys()))
sorted_rgbs = [cmap_dict[i] for i in sorted_keys]
print(sorted_rgbs)
print(sorted_keys)